In [1]:
CLUSTER_TYPE ='docker_cluster'
%env CLUSTER_TYPE $CLUSTER_TYPE
from dask.distributed import Client
client = Client('dask-scheduler:8786')
client

env: CLUSTER_TYPE=docker_cluster


<Client: 'tcp://172.20.0.2:8786' processes=4 threads=4, memory=3.81 GiB>

In [61]:
from time import time

from multiprocessing import cpu_count

import numpy as np
import pandas as pd
import sklearn.datasets
import matplotlib.pyplot as plt

import dask
import dask.array as da
import dask.dataframe as dd
from dask import compute
from dask import delayed
from dask.distributed import wait

import dask_ml.datasets
import dask_ml.cluster

import sklearn.cluster
##from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.utils.extmath import squared_norm

# Utils

In [33]:
#we want to work with dask array, hence we trasform everything to dask arrays
def make_da(X):
        if isinstance(X, dd.DataFrame):
            X = X.to_dask_array(lengths=True)
            
        if isinstance(X, da.core.Array): 
            return X
        
        else: raise ValueError("input is neither a dask array nor a dask dataframe")
        return X

In [ ]:
def squared_euclidean_distance(x_i, x_j):
    diff= (x_i, x_j)
    return da.dot(diff, diff.T)

In [ ]:
# we need to parallelize sklearn.metrics.pairwise_distances. 
# I didn't really did a general code as the one of sklearn.metrics.pairwise_distances but I directly compute the min
#distance of each point from the centroids and sum over all points 

def min_distance_point_from_set(point, Set):
    dist=[]
    for n in range(len(Set)):
        dist.append(delayed(squared_euclidean_distance)(point, Set[n]))
    min_dist=delayed(min)(dist)
    return min_dist 
# we only compute when we call compute(evaluate_cost(...))


def evaluate_cost(X, centroids):
    # (da.Array, np.array) -> float
    min_distances=[]
    for i in range(len(data)):
        min_distances.append(min_distance_point_from_set(X[i]), centroids)
    tot=delayed(sum)(min_distances)
    return tot.compute()

# Scalable K-means: K-means||

### Obeservation:
- Since the first centroid is random, it is a good choice to choose the first point of our dataset
- The centroids should be a numpy array stored in the machine. This way we can calculate distances without having to move data accross the nodes. This would have happened if the centroids array was distributed throughout the nodes.

In [ ]:
def k_means_scalable(X, k, l):
    X=make_da(X)
    n = X.shape[0] #we don't have to compute to get the distance
    iterations = 0
    centroids = da.compute(X[0, np.newaxis])[0] #since we computed it, it is a numpy array stored here, not on nodes.
    
    #c_idx = {idx}
    #(cost,) = compute(evaluate_cost(X, centers))